In [20]:
import os

import numpy as np

from scipy.optimize import minimize, fsolve

In [53]:
# User Inputs
#---------------------------------------------------------------------------------------#
FP_data_filename='Exp_FP_Data.csv'   # List of A,Z,y of measured fission products
U235f_filename=os.path.abspath(os.path.join(os.path.abspath(os.getcwd()), os.pardir))+'/Validation/U235f/Data/u235f_endf_cum_fy.csv'
U235h_filename=os.path.abspath(os.path.join(os.path.abspath(os.getcwd()), os.pardir))+'/Validation/U235h/Data/u235h_endf_cum_fy.csv'
U238f_filename=os.path.abspath(os.path.join(os.path.abspath(os.getcwd()), os.pardir))+'/Validation/U238f/Data/u238f_endf_cum_fy.csv'
U238h_filename=os.path.abspath(os.path.join(os.path.abspath(os.getcwd()), os.pardir))+'/Validation/U238h/Data/u238h_endf_cum_fy.csv'
#---------------------------------------------------------------------------------------#  

In [60]:
# Read Data
exp_data = np.loadtxt(open(FP_data_filename,"r"),delimiter=",")
A_exp=exp_data[:,0]
Z_exp=exp_data[:,1]
fy_exp=exp_data[:,2]
err_exp=exp_data[:,3]

fy_data=np.zeros((len(A_exp),8))  # [y_U235f,err,y_U235h,err,y_U238f,err,y_U238h,err]

# U235f Data
U235f_data = np.loadtxt(open(U235f_filename,"r"),delimiter=",")
Z=U235f_data[:,0]
A=U235f_data[:,1]
isomer=U235f_data[:,2]
y=U235f_data[:,3]*100
err=U235f_data[:,4]*100

for i in range(0,len(A)):
    if len(np.where(A_exp==A[i])[0])>0:
        ind=np.where(A_exp==A[i])[0][0]
        if A[i]==A_exp[ind] and Z[i]==Z_exp[ind] and isomer[i] == 0:
            fy_data[ind,0]=y[i]
            fy_data[ind,1]=err[i]

# U235h Data
U235h_data = np.loadtxt(open(U235h_filename,"r"),delimiter=",")
Z=U235h_data[:,0]
A=U235h_data[:,1]
isomer=U235h_data[:,2]
y=U235h_data[:,3]*100
err=U235h_data[:,4]*100

for i in range(0,len(A)):
    if len(np.where(A_exp==A[i])[0])>0:
        ind=np.where(A_exp==A[i])[0][0]
        if A[i]==A_exp[ind] and Z[i]==Z_exp[ind] and isomer[i] == 0:
            fy_data[ind,2]=y[i]
            fy_data[ind,3]=err[i]

# U238f Data
U238f_data = np.loadtxt(open(U238f_filename,"r"),delimiter=",")
Z=U238f_data[:,0]
A=U238f_data[:,1]
isomer=U238f_data[:,2]
y=U238f_data[:,3]*100
err=U238f_data[:,4]*100

for i in range(0,len(A)):
    if len(np.where(A_exp==A[i])[0])>0:
        ind=np.where(A_exp==A[i])[0][0]
        if A[i]==A_exp[ind] and Z[i]==Z_exp[ind] and isomer[i] == 0:
            fy_data[ind,4]=y[i]
            fy_data[ind,5]=err[i]

# U238h Data
U238h_data = np.loadtxt(open(U238h_filename,"r"),delimiter=",")
Z=U238h_data[:,0]
A=U238h_data[:,1]
isomer=U238h_data[:,2]
y=U238h_data[:,3]*100
err=U238h_data[:,4]*100

for i in range(0,len(A)):
    if len(np.where(A_exp==A[i])[0])>0:
        ind=np.where(A_exp==A[i])[0][0]
        if A[i]==A_exp[ind] and Z[i]==Z_exp[ind] and isomer[i] == 0:
            fy_data[ind,6]=y[i]
            fy_data[ind,7]=err[i]
        
print fy_data

[[  6.43197000e+00   6.43197000e-02   5.17353000e+00   1.44859000e-01
    5.14049000e+00   7.19669000e-02   4.89182000e+00   9.78364000e-02]
 [  6.00025000e+00   8.40035000e-02   5.14140000e+00   1.43959000e-01
    5.56226000e+00   7.78717000e-02   5.27609000e+00   7.38652000e-02]
 [  5.94313000e+00   8.32039000e-02   5.13560000e+00   1.43797000e-01
    6.16825000e+00   8.63555000e-02   5.70540000e+00   7.98757000e-02]
 [  4.24496000e-02   1.18859000e-03   1.07605000e+00   3.01294000e-02
    7.10025000e-02   1.42000000e-03   9.89295000e-01   2.77003000e-02]
 [  2.93477000e-02   3.22825000e-03   9.78732000e-01   1.07661000e-01
    4.15003000e-02   9.55000000e-03   8.22706000e-01   4.93623000e-02]
 [  3.10767000e-02   8.70148000e-04   1.09154000e+00   2.54188000e-02
    3.44372000e-02   6.89000000e-04   8.57043000e-01   3.14446000e-02]
 [  6.71794000e+00   4.29948000e+00   5.35629000e+00   3.42803000e+00
    6.75940000e+00   4.32601000e+00   5.99990000e+00   3.83993000e+00]
 [  5.9777300

In [65]:
# Chi-squared minimization function for a 4 fission split
def chisqfunc((a, b, c, d)):
    fy_model = abs(a)*fy_data[:,0] + abs(b)*fy_data[:,2] + abs(c)*fy_data[:,4] + abs(d)*fy_data[:,6]
    #print fy_model
    chisq = np.sum(((fy_exp - fy_model)/err_exp)**2)
    return chisq

In [62]:
# Perform minimization and output results
fs0 = np.array([0.8,0.1,0.05,0.05])
fiss_split =  minimize(chisqfunc, fs0, method='nelder-mead', \
                       options={'xtol': 1e-6, 'disp': True, 'maxiter': 5000,'maxfev':10000})
print fiss_split
print abs(fiss_split.x)
print abs(fiss_split.x)/np.sum(abs(fiss_split.x))
print np.sum(abs(fiss_split.x)/np.sum(abs(fiss_split.x)))

Optimization terminated successfully.
         Current function value: 1.746019
         Iterations: 256
         Function evaluations: 450
  status: 0
    nfev: 450
 success: True
     fun: 1.7460193485953137
       x: array([ 0.79612904,  0.17130491,  0.00781337,  0.02219318])
 message: 'Optimization terminated successfully.'
     nit: 256
[ 0.79612904  0.17130491  0.00781337  0.02219318]
[ 0.79817196  0.17174449  0.00783342  0.02225013]
1.0


In [64]:
# Python minimize performance
chisqfunc((0.79817196,0.17174449,0.00783342,0.02225013))

[ 6.17145462  5.83320363  5.80091511  0.24125587  0.21014674  0.23160961
  6.46843241  5.69129093  5.6580456   2.05279323  0.41432511  0.18026454
  0.02924528]


1.761007844961664

In [48]:
# CS performance
CS=(80.246817,16.537670,0.000670,2.939483)
chisqfunc(CS)
print CS/np.sum(CS)

[ 6.16086117  5.82040444  5.78623183  0.24109868  0.20959364  0.23064621
  6.45314991  5.67575161  5.64457851  2.04629431  0.41328834  0.17923989
  0.02918599]
[  8.04683948e-01   1.65833339e-01   6.71850006e-06   2.94759951e-02]
